# Malaria Detection using Deep Learning


https://www.kaggle.com/iarunava/cell-images-for-detecting-malaria

In [30]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [32]:
cp kaggle.json ~/.kaggle/

In [33]:
! chmod 600 ~/.kaggle/kaggle.json

In [34]:
!kaggle datasets download -d iarunava/cell-images-for-detecting-malaria

 99% 669M/675M [00:03<00:00, 219MB/s]
100% 675M/675M [00:03<00:00, 181MB/s]


In [35]:
!unzip cell-images-for-detecting-malaria.zip

Streaming output truncated to the last 5000 lines.
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_118.png  
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_126.png  
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_134.png  
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_141.png  
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_168.png  
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_175.png  
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_183.png  
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_221.png  
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_222.png  
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_87.png  
  inflating: cell_im

### Importing necessary libraries

In [36]:
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of layers
from tensorflow.keras.layers import Dense#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Flatten#used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D#Convolutional layer
from tensorflow.keras.layers import MaxPool2D# used for downsampling of the input
from tensorflow.keras.layers import Dropout #used for preventing overfitting of model
from tensorflow.keras.preprocessing.image import ImageDataGenerator# to artificially expand the size of a training dataset
import numpy as np#used for numerical analysis

In [37]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [38]:
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of layers
from tensorflow.keras.layers import Dense#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Flatten#used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D#Convolutional layer
from tensorflow.keras.layers import MaxPool2D# used for downsampling of the input
from tensorflow.keras.layers import Dropout #used for preventing overfitting of model
from tensorflow.keras.preprocessing.image import ImageDataGenerator# to artificially expand the size of a training dataset
import numpy as np#used for numerical analysis

In [39]:
img_width=64 #setting default image width size 
img_height=64 #setting default image height size

# SPLITTING DATA

In [40]:
pip install split-folders[full]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [41]:
import splitfolders

In [42]:
input_folder = 'cell_images'
splitfolders.ratio(input_folder, output="Dataset", 
                   seed=42, ratio=(.8, 0,.2), 
                   group_prefix=None)

Copying files: 27560 files [00:04, 5889.95 files/s]


### Image Data Agumentation

In [43]:
datagen=ImageDataGenerator(rescale=1/255.0,validation_split=0.2) #Image data augmentation

In [44]:
train_data_generator=datagen.flow_from_directory(directory='/content/Dataset/train',
                                                target_size=(img_width,img_height),class_mode="binary",
                                                batch_size=16,subset="training")#applying image data agumentation to training data
                                                
            
validation_data_generator=datagen.flow_from_directory(directory='/content/Dataset/test',
                                                target_size=(img_width,img_height),class_mode="binary",
                                                batch_size=16,subset="validation")#applying image data agumentation to testing data
                                                
                                              

Found 17638 images belonging to 3 classes.
Found 1102 images belonging to 3 classes.


In [45]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [46]:
train_data_generator.labels #checking the labels

array([0, 0, 0, ..., 1, 1, 1], dtype=int32)

### Creating the Model

In [47]:
model= Sequential() #our model

model.add(Conv2D(16,(3,3),input_shape=(img_width,img_height,3),activation="relu"))#convolutional layer
model.add(MaxPool2D(2,2)) # Downsamples the input
model.add(Dropout(0.2)) #drop input randomly for preventing from overfitting

model.add(Conv2D(32,(3,3),activation="relu"))
model.add(MaxPool2D(2,2))
model.add(Dropout(0.3))

model.add(Flatten())#flatten the dimension of the image
model.add(Dense(64,activation="relu"))#deeply connected neural network layer
model.add(Dropout(0.5))

model.add(Dense(1,activation="sigmoid")) #output layer with one neuron


In [48]:
model.summary() #summary of our neural network

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 16)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 31, 31, 16)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 14, 14, 32)        0

### Compiling the model

In [49]:
#compile modelb
model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])

### Fitting the model

In [51]:
history = model.fit_generator(generator=train_data_generator,
                              steps_per_epoch = len(train_data_generator),
                              epochs = 5,
                              validation_data = validation_data_generator,
                              validation_steps = len(validation_data_generator))

Epoch 1/5
   1/1103 [..............................] - ETA: 1:33 - loss: 0.1872 - accuracy: 0.9375

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """


1103/1103 [==============================] - 79s 72ms/step - loss: 0.1577 - accuracy: 0.9501 - val_loss: 0.1657 - val_accuracy: 0.9437
Epoch 2/5
1103/1103 [==============================] - 75s 68ms/step - loss: 0.1467 - accuracy: 0.9546 - val_loss: 0.2058 - val_accuracy: 0.9392
Epoch 3/5
1103/1103 [==============================] - 75s 68ms/step - loss: 0.1377 - accuracy: 0.9562 - val_loss: 0.2087 - val_accuracy: 0.9383
Epoch 4/5
1103/1103 [==============================] - 76s 69ms/step - loss: 0.1344 - accuracy: 0.9571 - val_loss: 0.2095 - val_accuracy: 0.9419
Epoch 5/5
1103/1103 [==============================] - 76s 68ms/step - loss: 0.1261 - accuracy: 0.9600 - val_loss: 0.1761 - val_accuracy: 0.9428


### Saving our model

In [52]:
from tensorflow.keras.models import load_model

model.save('maleria.h5')

### Predicting our results

In [53]:
from tensorflow.keras.models import load_model
model = load_model(r'/content/maleria.h5') #loading our trained model

In [54]:
y_pred = model.predict(validation_data_generator) #performing prediction to the test data

69/69 [==============================] - 2s 30ms/step


In [55]:
import numpy as np
y_pred = np.argmax(y_pred, axis=0)#Returns the indices of the maximum values along an axis
y_pred

array([383])

### Taking images as input an checking results

In [56]:
from tensorflow.keras.preprocessing import image
img=image.load_img(r'/content/Dataset/test/Parasitized/C100P61ThinF_IMG_20150918_144104_cell_168.png',target_size=(64,64))#loading image
x=image.img_to_array(img) #converting in to array formst
x.shape #checking its shape

(64, 64, 3)

In [57]:
x=np.expand_dims(x,axis=0) #changing its dimensions as per our requirement 
print(x.shape)
#img_data=x.reshape(1, 64, 64, 3)
#img_data.shape

(1, 64, 64, 3)


In [58]:
a=model.predict(x) #prediction

1/1 [==============================] - 0s 117ms/step


In [59]:
if(a==1):
    print("Uninfected")
else:
    print("Infected")

Infected
